ideas:

* only keep those in the NY state (188208 entries)
* index is very chaotic for the `Address City` column, many borough classification were put here in stead of in the `Address Borough` column
    * for example, there are 24236 entries of `Bronx` under `Address City` but only 16639 entries of `Bronx` under `Address Borough`. There may or may not be overlap but there are definitelly missing values.
    * we need to re-index our data
* there are a lot of blank values for `Address Borough`, maybe we can take those out after re-indexing.
* `Industry` is very nice becase there are no `NaN` values, however, there are industries with significantly less instances than others. 
    * resampling?
    * sample less from industries with more instances?
* `address zip` has several blank values, take out?
* since are target variable is the life span of the business, given the assumption that that businesses without a valid license are no longer operating, and that the time difference between license creation date and license expiration date indicates the lifespan of a business. We need to right censor instances whose license expires in the future because those expiration dates do not indicate the death of the business.


* extra feature we can calculate:
    * given a range of area from a particular business's longitude and latitude
        * the number of total other businesses in the area
        * the number of total other businesses in the area of the same industry type
    * convert date to numerical values and calculate the difference betweeen creation and expiration.
        * take note of businesses that expires after 2017, those need right censoring
    * number of schools given longitude and latitude
    * number of cafe given longitude and latitude

In [2]:
import pandas as pd

og_df = pd.read_csv('og_data_2017.csv', low_memory = False)

In [3]:
og_df

,LICENSE_NBR,LICENSE_TYPE,LIC_EXPIR_DD,LICENSE_STATUS,LICENSE_CREATE_DATE,INDUSTRY,BUSINESS_NAME,BUSINESS_NAME2,BUILDING,STREET,...,DETAIL,B1_X_COORD,B1_Y_COORD,BOROUGH_CODE,BBL,BIN,CENSUS_TRACT,COMMUNITY_BOARD,COUNCIL_DISTRICT,NTA
0,0501090-DCA,Business,12/31/2017,Active,1/11/1989,Commercial Lessor,RICHMOND HILL HALL CORPORATION,NaN,11709,HILLSIDE AVE,...,NaN,1030658.0,194427.0,4.0,4092570040,4193408,132,409.0,29.0,QN54
1,0501087-DCA,Business,12/31/2017,Active,10/25/1989,Commercial Lessor,FIFTH HALL CORP.,NaN,416,5TH AVE,...,NaN,988247.0,183392.0,3.0,3009980041,3324059,137,306.0,39.0,BK37
2,0501053-DCA,Business,12/31/2017,Active,10/25/1989,Commercial Lessor,"MASPETH HALL, INC.",NaN,6920,GRAND AVE,...,NaN,1013412.0,203884.0,4.0,4027910003,4062593,49302,405.0,30.0,QN30
3,0861918-DCA,Individual,10/31/2018,Active,12/22/1993,Tow Truck Driver,"YOON, HI B",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0888929-DCA,Business,8/1/2015,Inactive,1/18/2000,Pool or Billiard Room,STATUS Q BILLIARDS INC.,NaN,8218,3RD AVE,...,NaN,975934.0,167415.0,3.0,3060060044,3152045,62,310.0,43.0,BK31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114945,1006242-DCA,Business,12/31/2015,Inactive,4/7/1999,Laundry,"CRYSTAL CLEAN, LLC",NaN,1020,8TH AVE,...,NaN,989833.0,181548.0,3.0,3010940048,3337812,151,306.0,39.0,BK37
114946,2046952-DCA,Business,4/30/2018,Active,12/30/2016,Tow Truck Company,SUDDEN IMPACT AUTO BODY & TOWING INC,NaN,861,3RD AVE,...,Active Plate Numbers: 7838,983202.0,179025.0,3.0,3006760070,3010013,101,307.0,38.0,BK32
114947,2046935-DCA,Business,12/31/2018,Active,12/30/2016,Electronics Store,CELL NATION OF JAMAICA 112ST CORP,NaN,11112,JAMAICA AVE,...,NaN,1029455.0,193249.0,4.0,4093010005,4194835,124,409.0,28.0,QN54
114948,1005177-DCA,Business,3/31/2015,Inactive,3/23/1999,Garage,157 57TH THIRD CORP,NaN,153,E 57TH ST,...,"Vehicle Spaces: 38, Bicycle Spaces:",992941.0,216431.0,1.0,1013120028,1036911,11203,106.0,4.0,MN19


In [4]:
og_df['STATE'] == 'NY'

0         True
1         True
2         True
3         True
4         True
          ... 
114945    True
114946    True
114947    True
114948    True
114949    True
Name: STATE, Length: 114950, dtype: bool

In [5]:
og_df = og_df[og_df['STATE'] == 'NY']

# only keep businesses in NY

In [6]:
import numpy as np
og_df['B1_X_COORD']

0         1030658.0
1          988247.0
2         1013412.0
3               NaN
4          975934.0
            ...    
114945     989833.0
114946     983202.0
114947    1029455.0
114948     992941.0
114949          NaN
Name: B1_X_COORD, Length: 107983, dtype: float64

In [7]:
~np.isnan(og_df['B1_X_COORD'])

0          True
1          True
2          True
3         False
4          True
          ...  
114945     True
114946     True
114947     True
114948     True
114949    False
Name: B1_X_COORD, Length: 107983, dtype: bool

In [8]:
og_df = og_df [~ np.isnan(og_df['B1_X_COORD'])]
# only keep instances with longitude and latitude

og_df

,LICENSE_NBR,LICENSE_TYPE,LIC_EXPIR_DD,LICENSE_STATUS,LICENSE_CREATE_DATE,INDUSTRY,BUSINESS_NAME,BUSINESS_NAME2,BUILDING,STREET,...,DETAIL,B1_X_COORD,B1_Y_COORD,BOROUGH_CODE,BBL,BIN,CENSUS_TRACT,COMMUNITY_BOARD,COUNCIL_DISTRICT,NTA
0,0501090-DCA,Business,12/31/2017,Active,1/11/1989,Commercial Lessor,RICHMOND HILL HALL CORPORATION,NaN,11709,HILLSIDE AVE,...,NaN,1030658.0,194427.0,4.0,4092570040,4193408,132,409.0,29.0,QN54
1,0501087-DCA,Business,12/31/2017,Active,10/25/1989,Commercial Lessor,FIFTH HALL CORP.,NaN,416,5TH AVE,...,NaN,988247.0,183392.0,3.0,3009980041,3324059,137,306.0,39.0,BK37
2,0501053-DCA,Business,12/31/2017,Active,10/25/1989,Commercial Lessor,"MASPETH HALL, INC.",NaN,6920,GRAND AVE,...,NaN,1013412.0,203884.0,4.0,4027910003,4062593,49302,405.0,30.0,QN30
4,0888929-DCA,Business,8/1/2015,Inactive,1/18/2000,Pool or Billiard Room,STATUS Q BILLIARDS INC.,NaN,8218,3RD AVE,...,NaN,975934.0,167415.0,3.0,3060060044,3152045,62,310.0,43.0,BK31
5,0899195-DCA,Business,5/1/2016,Inactive,5/25/2000,Employment Agency,THE AGENCY GROUP U.S.A. LTD.,NaN,142,W 57TH ST,...,NaN,990027.0,218035.0,1.0,1010097501,1085969,137,105.0,4.0,MN17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114944,2046944-DCA,Business,3/31/2018,Active,12/30/2016,Stoop Line Stand,4125 MEAT CORP.,NaN,4125,102ND ST,...,"Product Category: Fruits, Vegetables, Soft Dri...",1022143.0,211798.0,4.0,4019760010,4048569,405,404.0,21.0,QN26
114945,1006242-DCA,Business,12/31/2015,Inactive,4/7/1999,Laundry,"CRYSTAL CLEAN, LLC",NaN,1020,8TH AVE,...,NaN,989833.0,181548.0,3.0,3010940048,3337812,151,306.0,39.0,BK37
114946,2046952-DCA,Business,4/30/2018,Active,12/30/2016,Tow Truck Company,SUDDEN IMPACT AUTO BODY & TOWING INC,NaN,861,3RD AVE,...,Active Plate Numbers: 7838,983202.0,179025.0,3.0,3006760070,3010013,101,307.0,38.0,BK32
114947,2046935-DCA,Business,12/31/2018,Active,12/30/2016,Electronics Store,CELL NATION OF JAMAICA 112ST CORP,NaN,11112,JAMAICA AVE,...,NaN,1029455.0,193249.0,4.0,4093010005,4194835,124,409.0,28.0,QN54


In [9]:
og_df_keepcol = pd.concat([og_df['LICENSE_NBR'], og_df['LIC_EXPIR_DD'],og_df['LICENSE_STATUS'],\
                           og_df['LICENSE_CREATE_DATE'],og_df['INDUSTRY'],og_df['ZIP'], og_df['B1_X_COORD'], \
                          og_df['B1_Y_COORD']], axis=1)
og_df_keepcol

,LICENSE_NBR,LIC_EXPIR_DD,LICENSE_STATUS,LICENSE_CREATE_DATE,INDUSTRY,ZIP,B1_X_COORD,B1_Y_COORD
0,0501090-DCA,12/31/2017,Active,1/11/1989,Commercial Lessor,11418,1030658.0,194427.0
1,0501087-DCA,12/31/2017,Active,10/25/1989,Commercial Lessor,11215,988247.0,183392.0
2,0501053-DCA,12/31/2017,Active,10/25/1989,Commercial Lessor,11378,1013412.0,203884.0
4,0888929-DCA,8/1/2015,Inactive,1/18/2000,Pool or Billiard Room,11209,975934.0,167415.0
5,0899195-DCA,5/1/2016,Inactive,5/25/2000,Employment Agency,10019,990027.0,218035.0
...,...,...,...,...,...,...,...,...
114944,2046944-DCA,3/31/2018,Active,12/30/2016,Stoop Line Stand,11368,1022143.0,211798.0
114945,1006242-DCA,12/31/2015,Inactive,4/7/1999,Laundry,11215,989833.0,181548.0
114946,2046952-DCA,4/30/2018,Active,12/30/2016,Tow Truck Company,11232,983202.0,179025.0
114947,2046935-DCA,12/31/2018,Active,12/30/2016,Electronics Store,11418,1029455.0,193249.0


In [10]:
clean_data = og_df_keepcol.dropna()
# drop any further NaN values

clean_data

,LICENSE_NBR,LIC_EXPIR_DD,LICENSE_STATUS,LICENSE_CREATE_DATE,INDUSTRY,ZIP,B1_X_COORD,B1_Y_COORD
0,0501090-DCA,12/31/2017,Active,1/11/1989,Commercial Lessor,11418,1030658.0,194427.0
1,0501087-DCA,12/31/2017,Active,10/25/1989,Commercial Lessor,11215,988247.0,183392.0
2,0501053-DCA,12/31/2017,Active,10/25/1989,Commercial Lessor,11378,1013412.0,203884.0
4,0888929-DCA,8/1/2015,Inactive,1/18/2000,Pool or Billiard Room,11209,975934.0,167415.0
5,0899195-DCA,5/1/2016,Inactive,5/25/2000,Employment Agency,10019,990027.0,218035.0
...,...,...,...,...,...,...,...,...
114944,2046944-DCA,3/31/2018,Active,12/30/2016,Stoop Line Stand,11368,1022143.0,211798.0
114945,1006242-DCA,12/31/2015,Inactive,4/7/1999,Laundry,11215,989833.0,181548.0
114946,2046952-DCA,4/30/2018,Active,12/30/2016,Tow Truck Company,11232,983202.0,179025.0
114947,2046935-DCA,12/31/2018,Active,12/30/2016,Electronics Store,11418,1029455.0,193249.0


In [12]:
clean_data['ZIP'] .value_counts().to_frame().reset_index()

,index,ZIP
0,11385,1008
1,10036,934
2,11220,933
3,11368,928
4,11214,810
...,...,...
480,6790,1
481,12203,1
482,10538,1
483,12566,1
